This will be about transferring Denis Charles example 3.2 of a distortion to a new one using isogenies.

In [1]:
###################### SETUP ######################
# E0 will be be the elliptic curve from Denis Charles example 3.2, i.e.  y^2 = x^3 - 35x + 98 over F_701
# E1 will be the elliptic Curve E0 -> E0/<Q> for a point Q of order 2
# We will be looking at the 5-torsion points and (5,2)=1
#
# phi : E_0 -> E_1 will be the 2-degree isogeny

 

p= 701
F= GF(p)
E0 = EllipticCurve(F,[0,0,0,-35,98])

#prints 2-torsion points of E0
E0(0).division_points(2) 

[(0 : 1 : 0), (319 : 0 : 1), (389 : 0 : 1), (694 : 0 : 1)]

In [2]:
# order 2 point of E0
Q = E0([319,0]) 

In [4]:
# Defining E1 as the isogeny phi: E_0 -> E_0/<Q>
# algorithm = 'factored' is in general faster for isogenies of prime power degree

phi = E0.isogeny(Q,algorithm='factored')
E1 = phi.codomain()
print(phi)

Composite morphism of degree 2 = 2:
  From: Elliptic Curve defined by y^2 = x^3 + 666*x + 98 over Finite Field of size 701
  To:   Elliptic Curve defined by y^2 = x^3 + 503*x + 66 over Finite Field of size 701


In [5]:
E0

Elliptic Curve defined by y^2 = x^3 + 666*x + 98 over Finite Field of size 701

In [6]:
E1

Elliptic Curve defined by y^2 = x^3 + 503*x + 66 over Finite Field of size 701

In [5]:
# Choose a 5-torsion point P_1 on E_1
# in order to use P_1 to check distortion

E1(0).division_points(5)
P1 = E1([22,7]) #5-torsion point on E1


In [6]:
# Q_0 is going to be a point on E_0, defined as the image of \hat{phi} (P_1)

Q0 = phi.dual()(P1) # image of phi_dual : E1 -> E0, mapping P1 to Q0
Q0

(135 : 309 : 1)

In [7]:
#Distort Q0
#See the distortion map \alpha of Denis Charles example 3.2

a = F(386)
x = 135
y = 309

dist_x = a^(-2)* (x-7*(1-a)^4/(x+a^2-2))  #1st coordinate of rational map
dist_y = a^(-3)*y * (1+ (7*(1-a)^4 / (x+a^2-2)^2 )) #2nd coordinate of rational map

Q0_dist = E0([dist_x,dist_y]) #distorted point (phi_dual(P1))
Q0_dist 


(469 : 81 : 1)

In [8]:
# Map distorted Q_0 back to E1
# P1_new = phi * distortion_of_ E0 * dual_phi (P1), recall P1 is 5-torsion on E_1

P1_new = phi(Q0_dist)
P1_new 



(644 : 337 : 1)

In [9]:
# Check we have distorted properly

n_torsion = 5
P1.weil_pairing(P1_new,n_torsion) #weil_pairing of P1 and its distorted version

210

In [10]:
P1_new.weil_pairing(P1,n_torsion)

464

Try to compose the whole thing into a single rational map to present as our distortion
Also, to generalize later to a single function with inputs:
E elliptic curve, P point to reduce modulo <P> for a new isogeny, a=(a_x,a_y) the distortion map on E
The function should be able to produce the new distortion map combining the rational maps of phi, phi_dual and a=(a_x,a_y)

In [11]:
phi.rational_maps()

((x^2 - 319*x + 313)/(x - 319), (x^2*y + 63*x*y - 197*y)/(x^2 + 63*x + 116))

In [12]:
phi.dual().rational_maps()

((-175*x^2 - 191*x - 52)/(x - 63),
 (263*x^2*y - 191*x*y + 84*y)/(x^2 - 126*x - 237))

In [13]:
############## SETUP ##############
# use a two variable polynomial ring for the rational maps to be defined over

K.<x,y> = PolynomialRing(F,2)

In [14]:
#  Define the coordinates of the rational maps from the isogeny phi and its dual

phi_x, phi_y = phi.rational_maps()
phi_dual_x, phi_dual_y = phi.dual().rational_maps()

In [15]:
# Define the distortion map in general as a fraction of elements in K = F_701[x,y]

dist_x = a^(-2)* (x-7*(1-a)^4/(x+a^2-2))    #a here is F(306), does not really matter here
dist_y = a^(-3)*y * (1+ (7*(1-a)^4 / (x+a^2-2)^2 ))


In [16]:
# Compose each coordinate as \phi \circ \alpha \circ \hat{\phi} : E_1 -> E_1

composition_x = phi_x( 
    x= dist_x( x = phi_dual_x, y = phi_dual_y),
    y = dist_y( x= phi_dual_x, y = phi_dual_y) 
    )

composition_y = phi_y( 
    x= dist_x( x = phi_dual_x, y = phi_dual_y),
    y = dist_y( x= phi_dual_x, y = phi_dual_y) 
    )


In [17]:
composition_x

(-201*x^8 - 211*x^7 - 187*x^6 + 135*x^5 + 309*x^4 + 234*x^3 - 183*x^2 - 242*x - 284)/(-296*x^7 - 234*x^6 + 276*x^5 + 189*x^4 - 14*x^3 + 151*x^2 - 276*x + 73)

In [18]:
composition_y

(156*x^12*y + 5*x^11*y - 41*x^10*y - 126*x^9*y - 194*x^8*y + 31*x^7*y + 153*x^6*y - 252*x^5*y + 317*x^4*y - 40*x^3*y - 52*x^2*y + 132*x*y + 311*y)/(167*x^12 - 98*x^11 + 102*x^10 - 218*x^9 - 12*x^8 - 253*x^7 + 155*x^6 + 57*x^5 - 269*x^4 + 9*x^3 - 254*x^2 - 297*x + 178)

Sanity check that the composed map works as distortion:

In [19]:
E1(0).division_points(5)

[(0 : 1 : 0),
 (22 : 7 : 1),
 (22 : 694 : 1),
 (207 : 158 : 1),
 (207 : 543 : 1),
 (324 : 228 : 1),
 (324 : 473 : 1),
 (339 : 315 : 1),
 (339 : 386 : 1),
 (439 : 14 : 1),
 (439 : 687 : 1),
 (494 : 111 : 1),
 (494 : 590 : 1),
 (538 : 265 : 1),
 (538 : 436 : 1),
 (619 : 51 : 1),
 (619 : 650 : 1),
 (640 : 117 : 1),
 (640 : 584 : 1),
 (644 : 337 : 1),
 (644 : 364 : 1),
 (667 : 21 : 1),
 (667 : 680 : 1),
 (675 : 16 : 1),
 (675 : 685 : 1)]

In [20]:
H = E1([675,16])
5*H == 0

True

In [21]:
H_dist_x = composition_x(675,16)
H_dist_y = composition_y(675,16)

H_dist = E1([H_dist_x, H_dist_y])
H_dist

(324 : 473 : 1)

In [34]:
5*H_dist

(0 : 1 : 0)

In [35]:
H.weil_pairing(H_dist,5)

1

In [36]:
H_dist.weil_pairing(H,5)

1

In [37]:
638^5 % 701

1

In [38]:
89^5 % 701

1

In [26]:
# Function distortion_new

p=701
F=GF(p)
K.<x,y> = PolynomialRing(F,2)


# Point Q must be coprime to degree of distortion map \alpha

def distortion_new(F, E, Q, a_x,a_y):
    phi = E.isogeny(Q,algorithm='factored')
    E_1 = phi.codomain()

    phi_x, phi_y = phi.rational_maps()
    phi_dual_x, phi_dual_y = phi.dual().rational_maps()

    
    composition_x = phi_x( 
        x = a_x( x = phi_dual_x, y = phi_dual_y),
        y = a_y( x = phi_dual_x, y = phi_dual_y) 
    )

    composition_y = phi_y( 
        x = a_x( x = phi_dual_x, y = phi_dual_y),
        y = a_y( x= phi_dual_x, y = phi_dual_y) 
    )
    return (E_1, phi, composition_x, composition_y)


In [27]:
##################### TEST FUNCTION  ###########################
b=F(386)
a_x = b^(-2)* (x-7*(1-b)^4/(x+b^2-2)) #1st coordinate of rational map
a_y = b^(-3)*y * (1+ (7*(1-b)^4 / (x+b^2-2)^2 )) #2nd coordinate of rational map

E0 = EllipticCurve(F,[0,0,0,-35,98])
Q = E0([319,0]) ### 2-torsion point yields 2-degree isogeny
n_torsion = 5
################################################################

E_1, phi, composition_x, composition_y = distortion_new(F, E0, Q, a_x,a_y)
n_torsion_E1 = E_1(0).division_points(n_torsion)



In [28]:
print(phi)
phi_x, phi_y = phi.rational_maps()
print(phi_x)
print(phi_y)

Composite morphism of degree 2 = 2:
  From: Elliptic Curve defined by y^2 = x^3 + 666*x + 98 over Finite Field of size 701
  To:   Elliptic Curve defined by y^2 = x^3 + 503*x + 66 over Finite Field of size 701
(x^2 - 319*x + 313)/(x - 319)
(x^2*y + 63*x*y - 197*y)/(x^2 + 63*x + 116)


In [30]:
E_1

Elliptic Curve defined by y^2 = x^3 + 503*x + 66 over Finite Field of size 701

In [29]:
# Exclude the identity element, the 0 corresponds to the list index not the elliptic curve 0 element

n_torsion_E1.pop(0) 

(0 : 1 : 0)

In [34]:
# Check that \hat{e}_5(P,P) is non-trivial for all P not 0:
# using the functions defined above
flag = True
for H in n_torsion_E1:  
        H_dist_x = composition_x(H[0],H[1])
        H_dist_y = composition_y(H[0],H[1])

        H_dist = E_1([H_dist_x, H_dist_y])
        if H.weil_pairing(H_dist,n_torsion) == 1:
            print( "It breaks at: ", H)
            flag = False
if(flag):
    print("All n-torsion points are properly distorted")

All n-torsion points are properly distorted
